In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datatable as dt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from datatable.models import Ftrl
import joblib

In [ ]:
http://localhost:5000

In [2]:
'''df_cat = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\test_categorical.csv",nrows=10,low_memory = False)
df_nume = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\test_numeric.csv",nrows=10,low_memory = False)
df_date = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\test_date.csv",nrows=10,low_memory = False)

joblib.dump(df_cat.columns,'pkl_files/categorical_columns')
joblib.dump(df_nume.columns,'pkl_files/numerical_columns')
joblib.dump(df_date.columns,'pkl_files/date_columns')'''

In [2]:
df = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\test_categorical.csv",nrows=100,low_memory = False)
columns = df.columns
lst = [0]*len(columns)

#This for loop counts the no of products passes through each sation
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_categorical.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    val = []
    columns = chunk_df.columns
    for i in columns:
        col = pd.Series(chunk_df[i])
        n = col.count()
        val.append(n)
    for i in range(len(lst)):
        lst[i] += val[i]
d = dict()#creating a dictionary where key is column name and value is the no of product passes through that line
for i in range(len(df.columns)):
    d[df.columns[i]] = lst[i]
imp_cols = []
for k,v in d.items():
    if v != 0:
        imp_cols.append(k)
len(imp_cols)

119it [02:53,  1.46s/it]


1978

In [18]:
a = df[list(imp_cols)]
a.shape

(100, 1978)

In [20]:
cat_data_cols = list(df.columns)
joblib.dump(cat_data_cols,'cat_data_cols.pkl')
joblib.dump(imp_cols,'cat_data_imp_cols.pkl')

['cat_data_imp_cols.pkl']

In [3]:
'''#extracting all categories from train data
all_categories = []
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_categorical.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    for i in range(len(chunk_df)):
        row = chunk_df.iloc[i]
        for j in row[1:]:
            if str(j) != 'nan':
                if j not in all_categories:
                    all_categories.append(j)
all_categories.append('not_passed')'''

"#extracting all categories from train data\nall_categories = []\nfor chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_categorical.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):\n    for i in range(len(chunk_df)):\n        row = chunk_df.iloc[i]\n        for j in row[1:]:\n            if str(j) != 'nan':\n                if j not in all_categories:\n                    all_categories.append(j)\nall_categories.append('not_passed')"

In [ ]:
'''#Numerical to categorical for train data
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
encoder.fit(np.array(all_categories).reshape(-1,1))'''

In [ ]:
#joblib.dump(encoder, 'OrdinalEncoder.pkl')

In [6]:
%%time
encoder = joblib.load('OrdinalEncoder.pkl')

Wall time: 1e+03 µs


In [7]:
#Numerical to categorical for train data
first_itr = None
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_categorical.csv.zip',usecols = imp_cols, iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    chunk_df = chunk_df.fillna('not_passed') #replacing all the nan values with not_passed category
    chunk_df = chunk_df.drop('Id',axis=1)
    for i in range(len(chunk_df)):
        row = encoder.transform(np.array(chunk_df.iloc[i]).reshape(-1,1))
        chunk_df.iloc[i] = row.reshape(1,-1)[0]
    chunk_df = chunk_df.astype('int32')
    if first_itr is None:
        chunk_dt = dt.Frame(chunk_df) #convertinf pandas dataframe to datatables
        first_itr = 'Not None'
    else:
        dt_ = dt.Frame(chunk_df)
        chunk_dt.rbind(dt_)#joining all the dataset

119it [38:11, 19.25s/it]


In [8]:
'''chunk_dt.to_csv("cat_to_nume.csv")'''

In [32]:
#joblib.dump(min_val,'min_val_each_col_in_cat_data.pkl')

['min_val_each_col_in_cat_data.pkl']

In [2]:
train = dt.fread('cat_to_nume.csv')

In [4]:
#normalizing the train data
from datatable import f, min, max
for col in tqdm(train.names):
    train[:,f[col]] = train[:, (f[col] - min(f[col]))/(max(f[col]) - min(f[col]))]

100%|██████████████████████████████████████████████████████████████████████████████| 1977/1977 [00:49<00:00, 40.02it/s]


In [5]:
train_num = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_numeric.csv',usecols = ['Response'])
train_num_ = dt.Frame(train_num)
target = train_num_[:,'Response']

In [6]:
X_train, X_valid = train[:800000, :], train[800000:, :]
y_train, y_valid = target[:800000, :], target[800000:, :]

In [6]:
model_ftrl = Ftrl(nepochs=5, nbins=10**8, lambda1=0.1,lambda2=1)
model_ftrl.fit(X_train, y_train, X_validation=X_valid, y_validation=y_valid)

FtrlFitOutput(epoch=2.0, loss=0.0006457398412749171)

In [7]:
y_pred = model_ftrl.predict(train) #predicting the result 
y_pred_csv = y_pred.to_pandas()

In [8]:
df_train = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_categorical.csv',usecols=["Id"])
y_pred_csv['Id'] = df_train["Id"]

In [9]:
y_pred_csv.to_csv("final_categorical_train.csv")

In [10]:
joblib.dump(model_ftrl, 'model_ftrl.pkl')

['model_ftrl.pkl']

In [3]:
#Train data
#creating some new engineered feature for categorical data
new_category_feature = pd.DataFrame()
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_categorical.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    df = chunk_df[['Id']].copy()
    df['no_of_category'] = chunk_df.count(axis=1)-1 #no of values except nan
    different_cat = []
    for i in range(len(chunk_df)):
        different_cat.append(len(chunk_df.iloc[i].value_counts())-1)
    df['different_category'] = different_cat #total no of different category used in a single product
    new_category_feature = pd.concat([new_category_feature,df])

119it [30:19, 15.29s/it]


In [8]:
probability_value = pd.read_csv('final_categorical_train.csv').drop("Unnamed: 0",axis=1)

In [10]:
new_category_feature['probability_value'] = probability_value['Response']

In [12]:
#new_category_feature.to_csv('final_df_train_categorical.csv')

# Numerical 

In [30]:
imp_numerical_features_name = ['L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F10',
       'L0_S0_F14', 'L0_S0_F20', 'L0_S1_F24', 'L0_S2_F48', 'L0_S3_F72',
       'L0_S3_F84', 'L0_S3_F92', 'L0_S3_F100', 'L0_S8_F144', 'L0_S9_F210',
       'L0_S10_F249', 'L0_S10_F259', 'L0_S12_F342', 'L0_S12_F350',
       'L0_S13_F356', 'L0_S17_F431', 'L0_S22_F596', 'L1_S24_F1490',
       'L1_S24_F1494', 'L1_S24_F1498', 'L1_S24_F1516', 'L1_S24_F1565',
       'L1_S24_F1571', 'L1_S24_F1575', 'L1_S24_F1578', 'L1_S24_F1581',
       'L1_S24_F1604', 'L1_S24_F1609', 'L1_S24_F1622', 'L1_S24_F1632',
       'L1_S24_F1657', 'L1_S24_F1672', 'L1_S24_F1695', 'L1_S24_F1713',
       'L1_S24_F1718', 'L1_S24_F1723', 'L1_S24_F1758', 'L1_S24_F1783',
       'L1_S24_F1831', 'L1_S24_F1842', 'L1_S24_F1844', 'L1_S24_F1846',
       'L1_S25_F2155', 'L1_S25_F2346', 'L1_S25_F2395', 'L1_S25_F2435',
       'L1_S25_F2443', 'L1_S25_F2449', 'L2_S26_F3040', 'L2_S26_F3051',
       'L2_S26_F3055', 'L2_S26_F3062', 'L2_S26_F3073', 'L2_S26_F3106',
       'L2_S27_F3144', 'L2_S27_F3199', 'L2_S27_F3210', 'L2_S27_F3214',
       'L3_S29_F3339', 'L3_S29_F3342', 'L3_S29_F3351', 'L3_S29_F3354',
       'L3_S29_F3357', 'L3_S29_F3373', 'L3_S29_F3382', 'L3_S29_F3407',
       'L3_S29_F3461', 'L3_S29_F3464', 'L3_S30_F3514', 'L3_S30_F3534',
       'L3_S30_F3544', 'L3_S30_F3554', 'L3_S30_F3574', 'L3_S30_F3584',
       'L3_S30_F3604', 'L3_S30_F3644', 'L3_S30_F3704', 'L3_S30_F3744',
       'L3_S30_F3754', 'L3_S30_F3759', 'L3_S30_F3794', 'L3_S30_F3799',
       'L3_S30_F3804', 'L3_S30_F3829', 'L3_S32_F3850', 'L3_S33_F3855',
       'L3_S33_F3857', 'L3_S33_F3859', 'L3_S33_F3865', 'L3_S36_F3920',
       'L3_S38_F3952', 'L3_S38_F3956', 'L3_S41_F4000', 'L3_S41_F4016',
       'L3_S49_F4211']

In [36]:
len(imp_numerical_features_name)

100

In [31]:
#joblib.dump(imp_numerical_features_name,'pkl_files/imp_numerical_features_name.pkl')

['pkl_files/imp_numerical_features_name.pkl']

In [32]:
imp_nume_col = joblib.load('pkl_files/imp_numerical_features_name.pkl')

In [34]:
df_nume = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_numeric.csv',usecols = imp_nume_col)

In [35]:
df_nume

,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F10,L0_S0_F14,L0_S0_F20,L0_S1_F24,L0_S2_F48,L0_S3_F72,...,L3_S33_F3855,L3_S33_F3857,L3_S33_F3859,L3_S33_F3865,L3_S36_F3920,L3_S38_F3952,L3_S38_F3956,L3_S41_F4000,L3_S41_F4016,L3_S49_F4211
0,0.030,-0.034,-0.197,-0.179,0.116,-0.032,-0.273,-0.271,0.001,NaN,...,-0.119,-0.102,-0.048,0.026,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.029,-0.026,-0.210,-0.047,NaN,NaN,NaN,NaN,NaN,NaN
2,0.088,0.086,0.003,-0.052,0.025,-0.072,0.250,0.057,-0.005,NaN,...,0.199,0.191,0.236,0.108,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.036,-0.064,0.294,0.330,0.161,0.128,-0.253,0.147,0.005,NaN,...,0.017,0.031,0.165,0.035,-0.030,NaN,NaN,NaN,NaN,NaN
4,-0.055,-0.086,0.294,0.330,0.025,0.168,0.042,-0.012,NaN,-0.272,...,-0.074,-0.049,0.147,-0.006,0.083,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183742,-0.023,0.033,-0.015,-0.016,0.070,-0.032,0.159,-0.084,0.002,NaN,...,-0.119,-0.095,0.152,0.035,NaN,NaN,NaN,NaN,NaN,NaN
1183743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.210,-0.184,-0.139,0.075,0.065,NaN,NaN,NaN,NaN,NaN
1183744,0.075,0.123,-0.343,-0.325,0.161,-0.232,0.046,0.057,NaN,-0.202,...,0.017,0.019,-0.114,-0.177,-0.036,NaN,NaN,NaN,NaN,NaN
1183745,-0.062,-0.019,-0.179,-0.179,0.116,-0.072,0.156,-0.010,NaN,-0.188,...,0.062,0.072,0.031,0.083,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
#df_nume.to_csv('final_df_train_numeric.csv')

# Date 

In [16]:
import joblib
imp_date_cols = joblib.load('pkl_files/imp_date_cols_name.pkl')

In [19]:
list(imp_date_cols.values)

['Unnamed: 0',
 'L0_S0_D1',
 'L0_S0_D3',
 'L0_S1_D26',
 'L0_S1_D30',
 'L0_S2_D34',
 'L0_S2_D38',
 'L0_S3_D70',
 'L0_S3_D74',
 'L0_S4_D106',
 'L0_S4_D111',
 'L0_S5_D115',
 'L0_S5_D117',
 'L0_S6_D120',
 'L0_S6_D124',
 'L0_S7_D137',
 'L0_S7_D139',
 'L0_S8_D145',
 'L0_S8_D147',
 'L0_S9_D152',
 'L0_S9_D162',
 'L0_S10_D216',
 'L0_S10_D226',
 'L0_S11_D280',
 'L0_S11_D284',
 'L0_S12_D331',
 'L0_S12_D333',
 'L0_S13_D355',
 'L0_S13_D357',
 'L0_S14_D360',
 'L0_S14_D364',
 'L0_S15_D395',
 'L0_S15_D398',
 'L0_S16_D423',
 'L0_S16_D428',
 'L0_S17_D432',
 'L0_S17_D434',
 'L0_S18_D437',
 'L0_S18_D441',
 'L0_S19_D454',
 'L0_S19_D456',
 'L0_S20_D462',
 'L0_S20_D464',
 'L0_S21_D469',
 'L0_S21_D479',
 'L0_S22_D543',
 'L0_S22_D553',
 'L0_S23_D617',
 'L0_S23_D621',
 'L1_S24_D1826',
 'L1_S24_D1828',
 'L1_S25_D1854',
 'L1_S25_D1857',
 'L2_S26_D3037',
 'L2_S26_D3041',
 'L2_S27_D3130',
 'L2_S27_D3134',
 'L2_S28_D3223',
 'L2_S28_D3227',
 'L3_S29_D3316',
 'L3_S29_D3319',
 'L3_S30_D3726',
 'L3_S30_D3731',
 'L3_S31_

In [10]:
st = []
imp_date_single_cols = []
for col in imp_date_cols[1:]:
    station = col.split("_")[1]
    if station not in st:
        st.append(station)
        imp_date_single_cols.append(col)

In [13]:
#joblib.dump(imp_date_single_cols,'imp_date_single_cols_name.pkl')

['imp_date_single_cols_name.pkl']

In [14]:
diff_st_date = []
for i in range(51):
    diff_st_date.append('S'+str(i))

In [18]:
ohe_station = pd.DataFrame(columns=diff_st_date)
df = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv',usecols = imp_date_single_cols)

In [19]:
leave_col = ['S24','S25']
for col in tqdm(df.columns):
    lst = [0]*len(df)
    st = col.split("_")[1]
    if st not in leave_col:
        temp = df[col]
        for i in range(len(temp)):
            val = temp.iloc[i]
            if str(val) != 'nan':
                lst[i] = 1
    ohe_station[st] = lst

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [04:57<00:00,  5.72s/it]


In [21]:
df_date = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv',nrows=10)
col_24 = []
col_25 = []
#doing one hot encoding of of S24 and S25 station 
for i in df_date.columns:
    if 'S24' in i.split("_"):
        col_24.append(i)
    if 'S25' in i.split("_"):
        col_25.append(i)

In [22]:
col_24_25 = [col_24,col_25]
joblib.dump(col_24_25,'col_24_25.pkl')

['col_24_25.pkl']

In [28]:
df_24 = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv',usecols = col_24)
df_25 = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv',usecols = col_25)

In [29]:
n_24 = [0]*len(df_24)
for i in tqdm(range(len(df_24))):
    row = df_24.iloc[i]
    if row.count() != 0:
        n_24[i] = 1
ohe_station['S24'] = n_24

n_25 = [0]*len(df_25)
for i in tqdm(range(len(df_25))):
    row = df_25.iloc[i]
    if row.count() != 0:
        n_25[i] = 1
ohe_station['S25'] = n_25

100%|█████████████████████████████████████████████████████████████████████| 1183747/1183747 [01:14<00:00, 15840.70it/s]


In [30]:
ohe_station

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S42,S43,S44,S45,S46,S47,S48,S49,S50,S51
0,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183742,1,1,1,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1183743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1183744,1,1,0,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1183745,1,1,0,1,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#ohe_station.to_csv("ohe_station.csv")

In [32]:
start_station = []
end_station = []
path_length = []
for i in tqdm(range(len(ohe_station))):
    row = ohe_station.iloc[i]
    start = 0
    end = 0
    
    for j in range(len(row)):
        if row[j] == 1:
            start = j
            break
    for k in range(len(row)-1,-1,-1):
        if row[k] == 1:
            end = k
            break
    path_length.append(np.count_nonzero(row == 1))
    start_station.append(start)
    end_station.append(end)

100%|██████████████████████████████████████████████████████████████████████| 1183747/1183747 [05:16<00:00, 3737.28it/s]


In [33]:
ohe_station['start_station'] = start_station
ohe_station['end_station'] = end_station
ohe_station['path_length'] = path_length

In [34]:
df = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv.zip',usecols = ['Id'])
ohe_station['Id'] = df['Id']
ohe_station.head(5)

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S46,S47,S48,S49,S50,S51,start_station,end_station,path_length,Id
0,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,37,14,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,12,37,13,6
2,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,37,13,7
3,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,37,13,9
4,1,1,0,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,37,13,11


In [35]:
#credit:-https://www.kaggle.com/choithuthoi/my-solution

new_date_feature = pd.DataFrame()
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    columns = chunk_df.columns[1:] 
    df = chunk_df[['Id']].copy()
    df['mintime'] = chunk_df[columns].min(axis=1).values
    df['maxtime'] = chunk_df[columns].max(axis=1).values
    df['time_difference'] =  df['maxtime'] - df['mintime']
    df.fillna(0,inplace=True)
    week_duration = 1679
    
    df['part_week'] = ((df['mintime'].values * 100)  % week_duration).astype(np.int64)
    df['min_time_station'] =  chunk_df[columns].idxmin(axis = 1).apply(lambda s: int(s.split('_')[1][1:]) if s is not np.nan else -1)
    df['max_time_station'] =  chunk_df[columns].idxmax(axis = 1).apply(lambda s: int(s.split('_')[1][1:]) if s is not np.nan else -1)
    new_date_feature = pd.concat([new_date_feature,df])

119it [02:14,  1.13s/it]


In [36]:
#credit:-https://www.kaggle.com/choithuthoi/my-solution
new_date_feature.sort_values(by=['mintime', 'Id'], inplace=True)
new_date_feature['mindate_id_diff'] = new_date_feature.Id.diff()
midr = np.full_like(new_date_feature.mindate_id_diff.values, np.nan)
midr[0:-1] = -new_date_feature.mindate_id_diff.values[1:]
new_date_feature['mindate_id_diff_reverse'] = midr

In [37]:
new_date_feature.sort_values(by=['Id'],inplace=True)

In [38]:
new_date_feature.head()

,Id,mintime,maxtime,time_difference,part_week,min_time_station,max_time_station,mindate_id_diff,mindate_id_diff_reverse
0,4,82.24,87.29,5.05,1508,0,35,-2314450.0,-224451.0
1,6,1313.12,1315.75,2.63,350,12,33,-943242.0,-67530.0
2,7,1618.70,1624.42,5.72,686,0,33,-2313512.0,-26876.0
3,9,1149.20,1154.16,4.96,748,0,36,-2196581.0,-7838.0
4,11,602.64,606.02,3.38,1499,0,36,-2226678.0,-6827.0


In [39]:
final_date_feature = pd.merge(new_date_feature,ohe_station,on = 'Id')
final_date_feature.head()

,Id,mintime,maxtime,time_difference,part_week,min_time_station,max_time_station,mindate_id_diff,mindate_id_diff_reverse,S0,...,S45,S46,S47,S48,S49,S50,S51,start_station,end_station,path_length
0,4,82.24,87.29,5.05,1508,0,35,-2314450.0,-224451.0,1,...,0,0,0,0,0,0,0,0,37,14
1,6,1313.12,1315.75,2.63,350,12,33,-943242.0,-67530.0,0,...,0,0,0,0,0,0,0,12,37,13
2,7,1618.70,1624.42,5.72,686,0,33,-2313512.0,-26876.0,1,...,0,0,0,0,0,0,0,0,37,13
3,9,1149.20,1154.16,4.96,748,0,36,-2196581.0,-7838.0,1,...,0,0,0,0,0,0,0,0,37,13
4,11,602.64,606.02,3.38,1499,0,36,-2226678.0,-6827.0,1,...,0,0,0,0,0,0,0,0,37,13


In [58]:
imp_date_cols_name = list(joblib.load('imp_date_cols_name.pkl').values[1:])
imp_date_cols_name.append("Id")
imp_date_feature = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv.zip',usecols = imp_date_cols_name)
imp_date_feature.head()

,Id,L0_S0_D1,L0_S0_D3,L0_S1_D26,L0_S1_D30,L0_S2_D34,L0_S2_D38,L0_S3_D70,L0_S3_D74,L0_S4_D106,...,L3_S47_D4140,L3_S47_D4145,L3_S48_D4194,L3_S48_D4195,L3_S49_D4218,L3_S49_D4223,L3_S50_D4242,L3_S50_D4244,L3_S51_D4255,L3_S51_D4257
0,4,82.24,82.24,82.24,82.24,82.24,82.24,NaN,NaN,82.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1149.20,1149.20,1149.20,1149.21,1149.21,NaN,NaN,1149.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,602.64,602.64,602.64,NaN,NaN,602.64,602.64,602.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
final_date_feature = pd.merge(final_date_feature,imp_date_feature,on='Id')

In [60]:
final_date_feature.head()

,Id,mintime,maxtime,time_difference,part_week,min_time_station,max_time_station,mindate_id_diff,mindate_id_diff_reverse,S0,...,L3_S47_D4140,L3_S47_D4145,L3_S48_D4194,L3_S48_D4195,L3_S49_D4218,L3_S49_D4223,L3_S50_D4242,L3_S50_D4244,L3_S51_D4255,L3_S51_D4257
0,4,82.24,87.29,5.05,1508,0,35,-2314450.0,-224451.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,1313.12,1315.75,2.63,350,12,33,-943242.0,-67530.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1624.42,5.72,686,0,33,-2313512.0,-26876.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1154.16,4.96,748,0,36,-2196581.0,-7838.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,606.02,3.38,1499,0,36,-2226678.0,-6827.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#final_date_feature.to_csv('final_date_feature_train1.csv')

In [2]:
final_date_feature = pd.read_csv('final_date_feature_train1.csv').drop('Unnamed: 0',axis=1)

In [3]:
df_date = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv',nrows=10)
line = ['L0','L1','L2','L3']
d = {} #key=station name and value=all the date feature related to that station
for l in line:
    lst = []
    for i in df_date.columns[1:]:
        t = i.split("_")
        if l in t:
            lst.append(i)
    d[l] = lst

In [4]:
#total no of feature related to each production line / how many features visited by the product
fraction_of_measurements = pd.DataFrame()
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    df = chunk_df[['Id']].copy()
    for l in line:
        col = d[l]
        temp = chunk_df[col]
        df['fraction_of_measurements'+l] = temp.count(axis=1)/len(col)
    fraction_of_measurements = pd.concat([fraction_of_measurements,df])

119it [01:23,  1.43it/s]


In [5]:
final_date_feature['average_time_per_measurement'] = final_date_feature['time_difference']/final_date_feature['path_length']

In [6]:
final_date_features = pd.merge(final_date_feature,fraction_of_measurements,on="Id")

In [7]:
final_date_features.shape

(1183747, 171)

In [8]:
imp_station = ['S32', 'S24', 'S38', 'S26', 'S28', 'S27', 'S36', 'S37', 'S30', 'S29']

In [9]:
lst = []
imp_date_st = final_date_features[imp_station]
for i in tqdm(range(len(final_date_features))):
    lst.append((np.count_nonzero(imp_date_st.iloc[i]))/10)

100%|█████████████████████████████████████████████████████████████████████| 1183747/1183747 [01:04<00:00, 18376.96it/s]


In [10]:
final_date_features['error_rate_on_imp_stations'] = lst

In [11]:
date = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv',nrows=10)
columns = date.columns

In [12]:
imp_st_col_date = []
for col in columns[1:]:
    if col.split("_")[1] in imp_station:
        imp_st_col_date.append(col)

In [13]:
imp_st_col_date.append("Id")

In [14]:
new_date_feature_imp_st = pd.DataFrame()
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv.zip',iterator=True, usecols = imp_st_col_date, chunksize=10000,compression='zip',low_memory=False)):
    columns = chunk_df.columns[1:] 
    df = chunk_df[['Id']].copy()
    df['mintime_imp_st'] = chunk_df[columns].min(axis=1).values
    df['maxtime_imp_st'] = chunk_df[columns].max(axis=1).values
    df['time_difference_imp_st'] =  df['maxtime_imp_st'] - df['mintime_imp_st']
    new_date_feature_imp_st = pd.concat([new_date_feature_imp_st,df])

119it [01:03,  1.87it/s]


In [15]:
df_ = pd.merge(final_date_features,new_date_feature_imp_st,on="Id")

In [16]:
df_.head(1)

,Id,mintime,maxtime,time_difference,part_week,min_time_station,max_time_station,mindate_id_diff,mindate_id_diff_reverse,S0,...,L3_S51_D4257,average_time_per_measurement,fraction_of_measurementsL0,fraction_of_measurementsL1,fraction_of_measurementsL2,fraction_of_measurementsL3,error_rate_on_imp_stations,mintime_imp_st,maxtime_imp_st,time_difference_imp_st
0,4,82.24,87.29,5.05,1508,0,35,-2314450.0,-224451.0,1,...,NaN,0.360714,0.255435,0.0,0.0,0.483516,0.3,87.26,87.29,0.03


In [18]:
total_no_of_different_date = []
for chunk_df in tqdm(pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_date.csv.zip',iterator=True, chunksize=10000,compression='zip',low_memory=False)):
    chunk_df.fillna(0,inplace=True)
    for i in range(len(chunk_df)):
        row = chunk_df.iloc[i][1:]
        total_no_of_different_date.append(len(pd.unique(row)))

119it [13:09,  6.63s/it]


In [19]:
df_['total_no_of_different_date'] = total_no_of_different_date

In [20]:
df_.shape

(1183747, 176)

In [22]:
df_.to_csv('final_df_train_date.csv')

In [23]:
df_cat = pd.read_csv('final_df_train_categorical.csv').drop('Unnamed: 0',axis=1)
df_nume = pd.read_csv('final_df_train_numeric.csv').drop('Unnamed: 0',axis=1)
df_date = pd.read_csv('final_df_train_date.csv').drop('Unnamed: 0',axis=1)

In [24]:
final_df = pd.merge(df_cat,df_nume,on="Id")
final_df = pd.merge(final_df,df_date,on='Id')

In [41]:
#final_df.to_csv('final_df_train.csv')

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import make_scorer, matthews_corrcoef
from datetime import datetime
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('final_df_train.csv',nrows = 300000).drop('Unnamed: 0',axis=1)
df.drop(['mindate_id_diff_reverse','mindate_id_diff'],axis=1,inplace=True)

In [3]:
X = df.drop(["Id","Response"], axis=1)
y = df['Response']

In [5]:
df = pd.read_csv('final_df_train.csv').drop('Unnamed: 0',axis=1)
df.drop(['mindate_id_diff_reverse','mindate_id_diff'],axis=1,inplace=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Id","Response"], axis=1),df["Response"],test_size=0.2, random_state=42, shuffle=True)

In [8]:
model_xgb=xgb.XGBClassifier(booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=3.2, learning_rate=0.5,
       max_delta_step=0, max_depth=3, min_child_weight=12, missing=1,
       n_estimators=65, n_jobs=1,random_state=0,
       objective='binary:logistic', reg_alpha=1.6,
       reg_lambda=0.2, scale_pos_weight=1, seed=None, silent=True,
       subsample=1,use_label_encoder=False)

In [9]:
model_xgb.fit(X_train,y_train)

[15:55:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:55:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=3.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=12, missing=1, monotone_constraints='()',
              n_estimators=65, n_jobs=1, num_parallel_tree=1, random_state=0,
              reg_alpha=1.6, reg_lambda=0.2, scale_pos_weight=1, seed=0,
              silent=True, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [14]:
y_pred_train = model_xgb.predict(X_train)
matthews_corrcoef(y_train,y_pred_train)

0.21913769914720096

In [15]:
y_pred_test = model_xgb.predict(X_test)
matthews_corrcoef(y_test,y_pred_test)

0.20136429126324365

In [20]:
import joblib
joblib.dump(model_xgb,'pkl_files/final_model.pkl')

['pkl_files/final_model.pkl']

In [2]:
import joblib
model = joblib.load('pkl_files/final_model.pkl')

In [20]:
df = pd.read_csv('final_df_train.csv',nrows=2500).drop('Unnamed: 0',axis=1)
df.drop(['mindate_id_diff_reverse','mindate_id_diff'],axis=1,inplace=True)

In [21]:
y = df['Response']
X= df.drop(['Response','Id'],axis=1)

In [22]:
y_pred = model.predict(X)

In [23]:
for i in range(len(y_pred)):
    if y_pred[i] == 1 and y[i] == 1:
        print(i)

2489
2490


In [24]:
matthews_corrcoef(y,y_pred)

0.4074303207171238

In [ ]:
'''a = pd.read_csv('C:\\OBS Studio\\self_case_study_file\\train_numeric.csv',nrows=5000,low_memory=False)
val = list(a.iloc[2490].values)
val[0] = int(val[0])
val = [str(i) for i in val]
str_ = ",".join(val)
file = open("newfile.txt",mode="w",encoding="utf-8")
file.seek(2490)
file.writelines(str_)
file.close'''

In [ ]:
'''input_file = open("input_data/categorical_data.txt","r")
input_ = list(input_file)
input_file.close()'''

In [ ]:
'''df = pd.read_csv('final_df_train.csv',nrows=5).drop('Unnamed: 0',axis=1)
df.drop(['mindate_id_diff_reverse','mindate_id_diff'],axis=1,inplace=True)

df_cat = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\train_categorical.csv",nrows=2500,low_memory = False)
df_nume = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\train_numeric.csv",nrows=2500,low_memory = False)
df_date = pd.read_csv("C:\\OBS Studio\\self_case_study_file\\train_date.csv",nrows=2500,low_memory = False)

df_cat.to_csv('input_data_df/cat_data.csv')
df_nume.to_csv('input_data_df/nume_data.csv')
df_date.to_csv('input_data_df/date_data.csv')'''